<a href="https://colab.research.google.com/github/obedjunias19/entailment-tree-reasoning/blob/main/csqa_2stage_llama70b_nshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle --upgrade --quiet

!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 140.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 17.4 MB/s eta 0:00:00


In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [3]:
!kaggle datasets download imjunias/csqa-logicalcombinations --unzip

Dataset URL: https://www.kaggle.com/datasets/imjunias/csqa-logicalcombinations
License(s): unknown
100% 5.90M/5.90M [00:00<00:00, 24.9MB/s]
100% 5.90M/5.90M [00:00<00:00, 21.2MB/s]


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [82]:
import json
import csv

# Read the JSONL file (one JSON object per line)
with open("/content/dev_all_hf.json", "r") as f:
    data = [json.loads(line) for line in f]

# Write to CSV
with open('dev_dataset.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)

    # Write header
    writer.writerow(['question', 'A', 'B', 'C', 'D', 'correct_label', 'correct_answer_text', 'qa_type'])

    # Write data rows
    for item in data:
        choices = item['choices']
        label = item['label']
        correct_answer = choices[label]

        # Map integer label to letter
        label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
        label_letter = label_map[label]

        correct_answer = choices[label]

        row = [
            item['question'],
            choices[0] if len(choices) > 0 else '',
            choices[1] if len(choices) > 1 else '',
            choices[2] if len(choices) > 2 else '',
            choices[3] if len(choices) > 3 else '',
            label_letter,  # Now shows A, B, C, or D
            correct_answer,
            item['qa_type']
        ]
        writer.writerow(row)

print("Conversion complete! Check dev_dataset.csv")




Conversion complete! Check dev_dataset.csv


In [83]:
qa_df = pd.read_csv('/content/dev_dataset.csv')
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text,qa_type
0,Where can you see a mountain in your own home?,virtual art display AND themed room decor,decorative sculpture AND landscape design book,themed room decor AND landscape design book,mountain-themed restaurant AND outdoor camping...,A,virtual art display AND themed room decor,AND
1,Why should people try learning language for bu...,nuanced market understanding AND casual conver...,effective collaboration AND nuanced market und...,strategic relationship building AND casual con...,nuanced market understanding AND personal enjo...,B,effective collaboration AND nuanced market und...,AND
2,It was something that had weighed on him his w...,moment of joy AND final answer,path to freedom AND temporary relief,path to freedom AND source of growth,turning point AND temporary relief,C,path to freedom AND source of growth,AND
3,The restaurant had no business out in the coun...,town center AND quiet lakeside,busy marketplace AND remote cabin,remote cabin AND industrial park,town center AND near public transport,D,town center AND near public transport,AND
4,What can a country do when it has jealousy but...,manipulate alliances AND conduct covert operat...,conduct covert operations AND express competit...,manipulate alliances AND express competitive s...,express competitive spirit AND celebrate natio...,A,manipulate alliances AND conduct covert operat...,AND


In [7]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 38.2 MB/s eta 0:00:00


In [8]:
import transformers
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [9]:
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get("HF_TOKEN")

In [10]:
from huggingface_hub import login
login(new_session=False)

In [11]:
# import pandas as pd
# import random
# import ast

# # Load your dataset
# df = pd.read_csv("/kaggle/working/logical_combinations_output.csv")  # replace with your file

# # Safe eval to ensure lists are properly loaded
# def safe_eval(val):
#     if isinstance(val, list):
#         return val
#     if pd.isna(val):
#         return []
#     if isinstance(val, str):
#         try:
#             return ast.literal_eval(val)
#         except:
#             return []
#     return []

# df['logical_combinations'] = df['logical_combinations'].apply(safe_eval)

# # Prepare storage for QA sets
# qa_sets = {
#     'AND': [],
#     'OR': [],
#     'NEITHER': [],
#     'Mixed': []
# }

# # Label rotation - separate rotator for each question type
# class LabelRotator:
#     def __init__(self, name):
#         self.name = name
#         self.labels = ['A', 'B', 'C', 'D']
#         self.current_index = 0

#     def get_next_label(self):
#         label = self.labels[self.current_index]
#         self.current_index = (self.current_index + 1) % len(self.labels)
#         # print(f"DEBUG {self.name}: Assigned label {label}, next will be {self.labels[self.current_index]}")
#         return label

# # Create separate rotators for each question type
# rotators = {
#     'AND': LabelRotator('AND'),
#     'OR': LabelRotator('OR'),
#     'NEITHER': LabelRotator('NEITHER'),
#     'Mixed': LabelRotator('Mixed')
# }

# def create_qa_item(question, correct_ans, incorrect_ans, qa_type):
#     """Helper function to create a QA item with proper label rotation"""

#     # Get correct label from the appropriate rotator
#     correct_label = rotators[qa_type].get_next_label()

#     # Ensure we have exactly 3 incorrect answers
#     if len(incorrect_ans) < 3:
#         print(f"Warning: Only {len(incorrect_ans)} incorrect answers available for {qa_type}")
#         return None

#     selected_incorrect = random.sample(incorrect_ans, 3)

#     # Create options array with 4 positions
#     options = [''] * 4

#     # Place correct answer at the designated position
#     correct_pos = rotators[qa_type].labels.index(correct_label)
#     options[correct_pos] = correct_ans

#     # Fill remaining positions with incorrect answers
#     incorrect_positions = [i for i in range(4) if i != correct_pos]
#     for i, pos in enumerate(incorrect_positions):
#         options[pos] = selected_incorrect[i]

#     return {
#         'question': question,
#         'A': options[0],
#         'B': options[1],
#         'C': options[2],
#         'D': options[3],
#         'correct_label': correct_label,
#         'correct_answer_text': correct_ans,
#         'qa_type': qa_type
#     }

# # Track counts for each type
# type_counts = {'AND': 0, 'OR': 0, 'NEITHER': 0, 'Mixed': 0}

# for idx, row in df.iterrows():
#     question = row['question']
#     combos = row['logical_combinations']

#     # Extract AND/OR/NEITHER correct and incorrect lists
#     and_correct = combos.get('AND_combinations', {}).get('correct', [])
#     and_incorrect = combos.get('AND_combinations', {}).get('incorrect', [])

#     or_correct = combos.get('OR_combinations', {}).get('correct', [])
#     or_incorrect = combos.get('OR_combinations', {}).get('incorrect', [])

#     neither_correct = combos.get('NEITHER_combinations', {}).get('correct', [])
#     neither_incorrect = combos.get('NEITHER_combinations', {}).get('incorrect', [])

#     # --- AND-only QA ---
#     if and_correct and len(and_incorrect) >= 3:
#         correct_ans = random.choice(and_correct)
#         qa_item = create_qa_item(question, correct_ans, and_incorrect, 'AND')
#         if qa_item:
#             qa_sets['AND'].append(qa_item)
#             type_counts['AND'] += 1
#             # print(f"AND Question {type_counts['AND']}: Label {qa_item['correct_label']}")

#     # --- OR-only QA ---
#     if or_correct and len(or_incorrect) >= 3:
#         correct_ans = random.choice(or_correct)
#         qa_item = create_qa_item(question, correct_ans, or_incorrect, 'OR')
#         if qa_item:
#             qa_sets['OR'].append(qa_item)
#             type_counts['OR'] += 1
#             # print(f"OR Question {type_counts['OR']}: Label {qa_item['correct_label']}")

#     # --- NEITHER-only QA ---
#     if neither_correct and len(neither_incorrect) >= 3:
#         correct_ans = random.choice(neither_correct)
#         qa_item = create_qa_item(question, correct_ans, neither_incorrect, 'NEITHER')
#         if qa_item:
#             qa_sets['NEITHER'].append(qa_item)
#             type_counts['NEITHER'] += 1
#             # print(f"NEITHER Question {type_counts['NEITHER']}: Label {qa_item['correct_label']}")

#     # --- Mixed QA ---
#     all_correct_types = [
#         ('AND', and_correct),
#         ('OR', or_correct),
#         ('NEITHER', neither_correct)
#     ]
#     all_incorrect_types = and_incorrect + or_incorrect + neither_incorrect

#     # Ensure at least one correct exists and enough incorrect answers
#     valid_categories = [cat for cat, lst in all_correct_types if lst]
#     if valid_categories and len(all_incorrect_types) >= 3:
#         chosen_category = random.choice(valid_categories)
#         chosen_correct_list = dict(all_correct_types)[chosen_category]
#         correct_ans = random.choice(chosen_correct_list)
#         qa_item = create_qa_item(question, correct_ans, all_incorrect_types, 'Mixed')
#         if qa_item:
#             qa_sets['Mixed'].append(qa_item)
#             type_counts['Mixed'] += 1
#             # print(f"Mixed Question {type_counts['Mixed']}: Label {qa_item['correct_label']}")

# print(f"\n=== GENERATION COMPLETE ===")
# for qtype, count in type_counts.items():
#     print(f"{qtype}: {count} questions generated")

# # --- Convert to DataFrames and save ---
# for key, qlist in qa_sets.items():
#     if qlist:  # Only save if we have questions
#         df_out = pd.DataFrame(qlist)
#         df_out.to_csv(f'QA_{key}.csv', index=False)
#         print(f"\n{key} QA saved: {len(df_out)} questions in QA_{key}.csv")

#         # Show label distribution for verification
#         label_counts = df_out['correct_label'].value_counts().sort_index()
#         print(f"  Label distribution: {dict(label_counts)}")

#         # Show label sequence for first 8 questions to verify rotation
#         print(f"  Label sequence (first 8): {list(df_out['correct_label'].head(8))}")
#     else:
#         print(f"{key}: No valid QA pairs generated")

# # Overall statistics
# total_questions = sum(len(qlist) for qlist in qa_sets.values())
# print(f"\nFinal total questions generated: {total_questions}")

In [84]:
# qa_df = pd.read_csv('/kaggle/input/csqa-pairs-baselines/QA_AND.csv')
qa_df.qa_type.unique()

array(['AND', 'OR', 'NEITHER', 'Mixed'], dtype=object)

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_qa_metrics(results_df):
    """

    Calculate comprehensive evaluation metrics for QA results

    Args:
        results_df: DataFrame with 'predicted_label' and 'correct_label' columns

    Returns:
        dict: Dictionary containing all evaluation metrics
    """

    # Remove None predictions for accurate calculation
    valid_predictions = results_df.dropna(subset=['predicted_label'])

    if len(valid_predictions) == 0:
        print("Warning: No valid predictions found!")
        return {}

    y_true = valid_predictions['correct_label'].tolist()
    y_pred = valid_predictions['predicted_label'].tolist()

    # Basic metrics
    total_questions = len(results_df)
    answered_questions = len(valid_predictions)
    unanswered_questions = total_questions - answered_questions

    # Accuracy (main metric for QA)
    accuracy = accuracy_score(y_true, y_pred)

    # Per-class metrics
    labels = ['A', 'B', 'C', 'D']
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )

    # Macro and micro averages
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    # Answer rate (percentage of questions answered)
    answer_rate = answered_questions / total_questions

    # Compile results
    metrics = {
        'total_questions': total_questions,
        'answered_questions': answered_questions,
        'unanswered_questions': unanswered_questions,
        'answer_rate': answer_rate,
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'micro_recall': micro_recall,
        'micro_f1': micro_f1,
    }

    # Per-class metrics
    for i, label in enumerate(labels):
        metrics[f'{label}_precision'] = precision[i]
        metrics[f'{label}_recall'] = recall[i]
        metrics[f'{label}_f1'] = f1[i]
        metrics[f'{label}_support'] = support[i]

    return metrics, cm, y_true, y_pred

def print_qa_metrics(metrics):
    """Pretty print the evaluation metrics"""
    print("=" * 50)
    print("QA EVALUATION METRICS")
    print("=" * 50)

    print(f"Total Questions: {metrics['total_questions']}")
    print(f"Answered Questions: {metrics['answered_questions']}")
    print(f"Answer Rate: {metrics['answer_rate']:.1%}")
    print(f"Accuracy: {metrics['accuracy']:.1%}")

    print("\n" + "-" * 30)
    print("OVERALL METRICS")
    print("-" * 30)
    print(f"Macro Precision: {metrics['macro_precision']:.3f}")
    print(f"Macro Recall: {metrics['macro_recall']:.3f}")
    print(f"Macro F1: {metrics['macro_f1']:.3f}")
    print(f"Micro Precision: {metrics['micro_precision']:.3f}")
    print(f"Micro Recall: {metrics['micro_recall']:.3f}")
    print(f"Micro F1: {metrics['micro_f1']:.3f}")

    print("\n" + "-" * 40)
    print("PER-CLASS METRICS")
    print("-" * 40)
    print(f"{'Class':<5} {'Precision':<9} {'Recall':<6} {'F1':<6} {'Support'}")
    print("-" * 40)

    for label in ['A', 'B', 'C', 'D']:
        precision = metrics[f'{label}_precision']
        recall = metrics[f'{label}_recall']
        f1 = metrics[f'{label}_f1']
        support = int(metrics[f'{label}_support'])
        print(f"{label:<5} {precision:<9.3f} {recall:<6.3f} {f1:<6.3f} {support}")

def plot_confusion_matrix(cm, labels=['A', 'B', 'C', 'D'], save_path=None):
    """Plot confusion matrix"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def analyze_answer_distribution(results_df):
    """Analyze the distribution of correct and predicted answers"""
    print("\n" + "=" * 50)
    print("ANSWER DISTRIBUTION ANALYSIS")
    print("=" * 50)

    # True label distribution
    true_dist = results_df['correct_label'].value_counts().sort_index()
    print("\nTrue Label Distribution:")
    for label in ['A', 'B', 'C', 'D']:
        count = true_dist.get(label, 0)
        pct = count / len(results_df) * 100
        print(f"  {label}: {count} ({pct:.1f}%)")

    # Predicted label distribution (excluding None)
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        pred_dist = valid_preds['predicted_label'].value_counts().sort_index()
        print("\nPredicted Label Distribution:")
        for label in ['A', 'B', 'C', 'D']:
            count = pred_dist.get(label, 0)
            pct = count / len(valid_preds) * 100
            print(f"  {label}: {count} ({pct:.1f}%)")

    # Check for bias towards certain options
    print("\nLabel Rotation Check:")
    if len(set(true_dist.values)) == 1:
        print("✓ Perfect label rotation detected")
    else:
        print("⚠ Uneven label distribution detected")

def comprehensive_qa_evaluation(results_df, save_plots=True):
    """Run complete evaluation pipeline"""

    # Calculate metrics
    metrics, cm, y_true, y_pred = calculate_qa_metrics(results_df)

    if not metrics:
        return

    # Print metrics
    print_qa_metrics(metrics)

    # Analyze distributions
    analyze_answer_distribution(results_df)

    # Plot confusion matrix
    if save_plots:
        plot_confusion_matrix(cm, save_path='confusion_matrix.png')
    else:
        plot_confusion_matrix(cm)

    # Print sklearn classification report for additional insights
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        print("\n" + "=" * 50)
        print("DETAILED CLASSIFICATION REPORT")
        print("=" * 50)
        print(classification_report(
            valid_preds['correct_label'],
            valid_preds['predicted_label'],
            labels=['A', 'B', 'C', 'D']
        ))

    return metrics

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Llama-3.3-70B-Instruct"

# 1) Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 2) Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# 3) Load 70B model entirely on GPU
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,  # official way
    device_map={"": 0},              # load FULL MODEL on GPU 0
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)

print("Model fully loaded on A100 80GB (4-bit NF4)!")


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00029-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00030-of-00030.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model fully loaded on A100 80GB (4-bit NF4)!


In [43]:
import torch
import re
from tqdm import tqdm

def batched_zero_shot_qa(model, tokenizer, qa_dataframe, batch_size=32):

    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain columns: {required_cols}")

    valid_choices = {"A", "B", "C", "D"}

    tokenizer.padding_side = "left"
    results = []

    # Split into batches
    num_samples = len(qa_dataframe)
    num_batches = (num_samples + batch_size - 1) // batch_size

    for b in tqdm(range(num_batches), desc="Running batched inference"):

        batch = qa_dataframe.iloc[b * batch_size: (b + 1) * batch_size]

        # ---- 1. Construct prompts for the whole batch ----
        prompts = []
        for _, row in batch.iterrows():
            prompt = f"""Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer.
                Do not include any other text, punctuation, or explanation.

    Question: {str(row['question'])}

    (A) {str(row['A'])}
    (B) {str(row['B'])}
    (C) {str(row['C'])}
    (D) {str(row['D'])}

    Answer:""".strip()
            prompts.append(prompt)

         # LEFT padding (important!)
        encoded = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        with torch.no_grad():
            generated = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=2,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        for i in range(len(batch)):
            # compute true input length based on left padding
            input_len = (attention_mask[i] == 1).sum().item()

            # extract continuation from END, not start
            gen_tokens = generated[i][input_ids.shape[1]:]  # simpler + reliable

            gen_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
            cleaned = (
                gen_text.strip()
                .upper()
                .replace('.', '')
                .replace(',', '')
                .replace(' ', '')
            )

            pred = cleaned[0] if cleaned and cleaned[0] in valid_choices else None

            row = batch.iloc[i]

            results.append({
                "question": row["question"],
                "predicted_label": pred,
                "correct_label": str(row["correct_label"]).strip().upper(),
                'full_qa': f"""Question: {str(row['question'])} \n
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}"""
            })

    return results


In [102]:
import re


def three_shot_and_qa(model, tokenizer, qa_dataframe, batch_size=32):

    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain columns: {required_cols}")

    valid_choices = {"A", "B", "C", "D"}

    tokenizer.padding_side = "left"
    results = []

    # Split into batches
    num_samples = len(qa_dataframe)
    num_batches = (num_samples + batch_size - 1) // batch_size

    for b in tqdm(range(num_batches), desc="Running batched inference"):

        batch = qa_dataframe.iloc[b * batch_size: (b + 1) * batch_size]

        # ---- 1. Construct prompts for the whole batch ----
        prompts = []
        for _, row in batch.iterrows():
            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.

                Examples:
                Question: Sammy wanted to go to where the people were. Where might he go?
                        A. social venues AND quiet retreats
                        B. local events AND social venues
                        C. sports arenas AND quiet retreats
                        D. sports arenas AND train platforms

                        Answer: B

                Question: The fox walked from the city into the forest, what was it looking for?
                        A. suitable prey AND shelter from disturbances
                        B. urban garden AND farm fields
                        C. natural habitat AND farm fields
                        D. suitable prey AND neighborhood pets

                        Answer: A

                Question: What home entertainment equipment requires cable?
                        A. wireless speaker AND portable projector
                        B. television AND home theater system
                        C. wireless speaker AND gaming console
                        D. digital frame AND portable projector

                        Answer: B

                 Question: {str(row['question'])}
                (A) {str(row['A'])}
                (B) {str(row['B'])}
                (C) {str(row['C'])}
                (D) {str(row['D'])}
                Answer:""".strip()
            prompts.append(prompt)

         # LEFT padding (important!)
        encoded = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        with torch.no_grad():
            generated = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=2,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        for i in range(len(batch)):
            # compute true input length based on left padding
            input_len = (attention_mask[i] == 1).sum().item()

            # extract continuation from END, not start
            gen_tokens = generated[i][input_ids.shape[1]:]  # simpler + reliable

            gen_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
            cleaned = (
                gen_text.strip()
                .upper()
                .replace('.', '')
                .replace(',', '')
                .replace(' ', '')
            )

            pred = cleaned[0] if cleaned and cleaned[0] in valid_choices else None

            row = batch.iloc[i]

            results.append({
                "question": row["question"],
                "predicted_label": pred,
                "correct_label": str(row["correct_label"]).strip().upper(),
                'full_qa': f"""Question: {str(row['question'])} \n
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}"""
            })

    return results

In [110]:
qa_df_AND = qa_df[qa_df.qa_type == 'AND']
results = batched_zero_shot_qa(model,tokenizer, qa_df_AND)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_AND_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)

for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted anwer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Running batched inference: 100%|██████████| 47/47 [02:59<00:00,  3.82s/it]

({'total_questions': 1500, 'answered_questions': 1403, 'unanswered_questions': 97, 'answer_rate': 0.9353333333333333, 'accuracy': 0.7940128296507484, 'macro_precision': 0.8016746792421636, 'macro_recall': 0.7929376549592622, 'macro_f1': 0.7943098235114389, 'micro_precision': 0.7940128296507484, 'micro_recall': 0.7940128296507484, 'micro_f1': 0.7940128296507484, 'A_precision': np.float64(0.8707482993197279), 'A_recall': np.float64(0.757396449704142), 'A_f1': np.float64(0.810126582278481), 'A_support': np.int64(338), 'B_precision': np.float64(0.7279236276849642), 'B_recall': np.float64(0.8495821727019499), 'B_f1': np.float64(0.7840616966580977), 'B_support': np.int64(359), 'C_precision': np.float64(0.827922077922078), 'C_recall': np.float64(0.7369942196531792), 'C_f1': np.float64(0.7798165137614679), 'C_support': np.int64(346), 'D_precision': np.float64(0.7801047120418848), 'D_recall': np.float64(0.8277777777777777), 'D_f1': np.float64(0.8032345013477089), 'D_support': np.int64(360)}, ar

In [104]:

def three_shot_or_qa(model, tokenizer, qa_dataframe,batch_size=32):

    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain columns: {required_cols}")

    valid_choices = {"A", "B", "C", "D"}

    tokenizer.padding_side = "left"
    results = []

    # Split into batches
    num_samples = len(qa_dataframe)
    num_batches = (num_samples + batch_size - 1) // batch_size

    for b in tqdm(range(num_batches), desc="Running batched inference"):

        batch = qa_dataframe.iloc[b * batch_size: (b + 1) * batch_size]

        # ---- 1. Construct prompts for the whole batch ----
        prompts = []
        for _, row in batch.iterrows():
            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.

                Examples:
                Question: Sammy wanted to go to where the people were. Where might he go?
                A. quiet retreats OR empty parks
                B. local events OR empty parks
                C. sports arenas OR empty parks
                D. train platforms OR empty parks

                Answer: B

                Question: The forgotten leftovers had gotten quite old, he found it covered in mold in the back of his what?
                A. living room OR dining area
                B. utility drawer OR kitchen counter
                C. living room OR utility drawer
                D. cold storage OR utility drawer

                Answer: D

                Question: The forgotten leftovers had gotten quite old, he found it covered in mold in the back of his what?
                A. living room OR dining area
                B. utility drawer OR kitchen counter
                C. living room OR utility drawer
                D. cold storage OR utility drawer

                Answer: D

                Now answer the following question:

                 Question: {str(row['question'])}
                (A) {str(row['A'])}
                (B) {str(row['B'])}
                (C) {str(row['C'])}
                (D) {str(row['D'])}
                Answer:""".strip()
            prompts.append(prompt)

         # LEFT padding (important!)
        encoded = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        with torch.no_grad():
            generated = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=2,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        for i in range(len(batch)):
            # compute true input length based on left padding
            input_len = (attention_mask[i] == 1).sum().item()

            # extract continuation from END, not start
            gen_tokens = generated[i][input_ids.shape[1]:]  # simpler + reliable

            gen_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
            cleaned = (
                gen_text.strip()
                .upper()
                .replace('.', '')
                .replace(',', '')
                .replace(' ', '')
            )

            pred = cleaned[0] if cleaned and cleaned[0] in valid_choices else None

            row = batch.iloc[i]

            results.append({
                "question": row["question"],
                "predicted_label": pred,
                "correct_label": str(row["correct_label"]).strip().upper(),
                'full_qa': f"""Question: {str(row['question'])} \n
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}"""
            })

    return results

In [111]:
qa_df_OR = qa_df[qa_df.qa_type == 'OR']
results = batched_zero_shot_qa(model, tokenizer, qa_df_OR)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_OR_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Running batched inference: 100%|██████████| 47/47 [02:58<00:00,  3.81s/it]

({'total_questions': 1500, 'answered_questions': 1295, 'unanswered_questions': 205, 'answer_rate': 0.8633333333333333, 'accuracy': 0.7644787644787645, 'macro_precision': 0.7762092196583616, 'macro_recall': 0.7614049325759471, 'macro_f1': 0.7636110247235428, 'micro_precision': 0.7644787644787645, 'micro_recall': 0.7644787644787645, 'micro_f1': 0.7644787644787645, 'A_precision': np.float64(0.8368055555555556), 'A_recall': np.float64(0.7507788161993769), 'A_f1': np.float64(0.7914614121510674), 'A_support': np.int64(321), 'B_precision': np.float64(0.7634408602150538), 'B_recall': np.float64(0.8352941176470589), 'B_f1': np.float64(0.797752808988764), 'B_support': np.int64(340), 'C_precision': np.float64(0.8312236286919831), 'C_recall': np.float64(0.652317880794702), 'C_f1': np.float64(0.7309833024118738), 'C_support': np.int64(302), 'D_precision': np.float64(0.6733668341708543), 'D_recall': np.float64(0.8072289156626506), 'D_f1': np.float64(0.7342465753424657), 'D_support': np.int64(332)}, 

In [106]:
def three_shot_nnor_qa(model, tokenizer, qa_dataframe,batch_size=32):

    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain columns: {required_cols}")

    valid_choices = {"A", "B", "C", "D"}

    tokenizer.padding_side = "left"
    results = []

    # Split into batches
    num_samples = len(qa_dataframe)
    num_batches = (num_samples + batch_size - 1) // batch_size

    for b in tqdm(range(num_batches), desc="Running batched inference"):

        batch = qa_dataframe.iloc[b * batch_size: (b + 1) * batch_size]

        # ---- 1. Construct prompts for the whole batch ----
        prompts = []
        for _, row in batch.iterrows():
            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.

                Examples:
                Question: Sammy wanted to go to where the people were. Where might he go?
                A. NEITHER local events NOR train platforms
                B. NEITHER sports arenas NOR train platforms
                C. NEITHER social venues NOR sports arenas
                D. NEITHER social venues NOR quiet retreats

                Answer: B

                Question: The forgotten leftovers had gotten quite old, he found it covered in mold in the back of his what?
                A. NEITHER cold storage NOR dining area
                B. NEITHER sealed container NOR living room
                C. NEITHER food cabinet NOR utility drawer
                D. NEITHER living room NOR utility drawer

                Answer: D

                Question: The only baggage the woman checked was a drawstring bag, where was she heading with it?
                A. NEITHER family gathering NOR local gathering
                B. NEITHER airport travel NOR local gathering
                C. NEITHER local gathering NOR short trip
                D. NEITHER business engagement NOR local gathering

                Answer: C

                Now answer the following question:

                 Question: {str(row['question'])}
                (A) {str(row['A'])}
                (B) {str(row['B'])}
                (C) {str(row['C'])}
                (D) {str(row['D'])}
                Answer:""".strip()
            prompts.append(prompt)

         # LEFT padding (important!)
        encoded = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        with torch.no_grad():
            generated = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=2,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        for i in range(len(batch)):
            # compute true input length based on left padding
            input_len = (attention_mask[i] == 1).sum().item()

            # extract continuation from END, not start
            gen_tokens = generated[i][input_ids.shape[1]:]  # simpler + reliable

            gen_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
            cleaned = (
                gen_text.strip()
                .upper()
                .replace('.', '')
                .replace(',', '')
                .replace(' ', '')
            )

            pred = cleaned[0] if cleaned and cleaned[0] in valid_choices else None

            row = batch.iloc[i]

            results.append({
                "question": row["question"],
                "predicted_label": pred,
                "correct_label": str(row["correct_label"]).strip().upper(),
                'full_qa': f"""Question: {str(row['question'])} \n
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}"""
            })

    return results

In [112]:
qa_df_NEITHER = qa_df[qa_df.qa_type == 'NEITHER']
results = batched_zero_shot_qa(model,tokenizer, qa_df_NEITHER, 32)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_NEITHER_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Running batched inference: 100%|██████████| 47/47 [03:07<00:00,  4.00s/it]

({'total_questions': 1500, 'answered_questions': 1039, 'unanswered_questions': 461, 'answer_rate': 0.6926666666666667, 'accuracy': 0.10202117420596728, 'macro_precision': 0.11618883626911378, 'macro_recall': 0.10755318602082692, 'macro_f1': 0.09212022170007966, 'micro_precision': 0.10202117420596728, 'micro_recall': 0.10202117420596728, 'micro_f1': 0.10202117420596728, 'A_precision': np.float64(0.18), 'A_recall': np.float64(0.03237410071942446), 'A_f1': np.float64(0.054878048780487805), 'A_support': np.int64(278), 'B_precision': np.float64(0.13805970149253732), 'B_recall': np.float64(0.13602941176470587), 'B_f1': np.float64(0.13703703703703704), 'B_support': np.int64(272), 'C_precision': np.float64(0.05235602094240838), 'C_recall': np.float64(0.03759398496240601), 'C_f1': np.float64(0.0437636761487965), 'C_support': np.int64(266), 'D_precision': np.float64(0.09433962264150944), 'D_recall': np.float64(0.2242152466367713), 'D_f1': np.float64(0.13280212483399734), 'D_support': np.int64(22

In [108]:
def three_shot_mixed_qa(model, tokenizer, qa_dataframe,batch_size=32):

    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain columns: {required_cols}")

    valid_choices = {"A", "B", "C", "D"}

    tokenizer.padding_side = "left"
    results = []

    # Split into batches
    num_samples = len(qa_dataframe)
    num_batches = (num_samples + batch_size - 1) // batch_size

    for b in tqdm(range(num_batches), desc="Running batched inference"):

        batch = qa_dataframe.iloc[b * batch_size: (b + 1) * batch_size]

        # ---- 1. Construct prompts for the whole batch ----
        prompts = []
        for _, row in batch.iterrows():
            prompt = f"""
               Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.

                Examples:
                   Question: The fox walked from the city into the forest, what was it looking for?

                    A. natural habitat AND shelter from disturbances
                    B. farm fields OR neighborhood pets
                    C. city park AND neighborhood pets
                    D. farm fields AND neighborhood pets

                Answer: A

                Question: Google Maps and other highway and street GPS services have replaced what?
                    A. historical navigation logs AND tourist information centers
                    B. NEITHER printed road maps NOR route planning software
                    C. manual navigation techniques AND tourist information centers
                    D. manual navigation techniques AND route planning software

                    Answer: D

                Question: Where is a business restaurant likely to be located?
                    A. corporate office clusters AND at university campuses
                    B. business park entrances AND near conference venues
                    C. NEITHER near conference venues NOR at university campuses
                    D. NEITHER near conference venues NOR near tourist sites

                    Answer: B

                Now answer the following question:

                 Question: {str(row['question'])}
                (A) {str(row['A'])}
                (B) {str(row['B'])}
                (C) {str(row['C'])}
                (D) {str(row['D'])}
                Answer:""".strip()
            prompts.append(prompt)

         # LEFT padding (important!)
        encoded = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        with torch.no_grad():
            generated = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=2,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        for i in range(len(batch)):
            # compute true input length based on left padding
            input_len = (attention_mask[i] == 1).sum().item()

            # extract continuation from END, not start
            gen_tokens = generated[i][input_ids.shape[1]:]  # simpler + reliable

            gen_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)
            cleaned = (
                gen_text.strip()
                .upper()
                .replace('.', '')
                .replace(',', '')
                .replace(' ', '')
            )

            pred = cleaned[0] if cleaned and cleaned[0] in valid_choices else None

            row = batch.iloc[i]

            results.append({
                "question": row["question"],
                "predicted_label": pred,
                "correct_label": str(row["correct_label"]).strip().upper(),
                'full_qa': f"""Question: {str(row['question'])} \n
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}"""
            })

    return results

In [113]:
qa_df_MIXED = qa_df[qa_df.qa_type == 'Mixed']
results = batched_zero_shot_qa(model,tokenizer, qa_df_MIXED)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_MIXED_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Running batched inference: 100%|██████████| 47/47 [03:02<00:00,  3.88s/it]

({'total_questions': 1500, 'answered_questions': 1300, 'unanswered_questions': 200, 'answer_rate': 0.8666666666666667, 'accuracy': 0.5784615384615385, 'macro_precision': 0.5892739886455518, 'macro_recall': 0.5774252543812874, 'macro_f1': 0.577850233287701, 'micro_precision': 0.5784615384615385, 'micro_recall': 0.5784615384615385, 'micro_f1': 0.5784615384615385, 'A_precision': np.float64(0.6571428571428571), 'A_recall': np.float64(0.5015576323987538), 'A_f1': np.float64(0.568904593639576), 'A_support': np.int64(321), 'B_precision': np.float64(0.5263157894736842), 'B_recall': np.float64(0.6060606060606061), 'B_f1': np.float64(0.5633802816901409), 'B_support': np.int64(330), 'C_precision': np.float64(0.6263736263736264), 'C_recall': np.float64(0.5394321766561514), 'C_f1': np.float64(0.5796610169491525), 'C_support': np.int64(317), 'D_precision': np.float64(0.5472636815920398), 'D_recall': np.float64(0.6626506024096386), 'D_f1': np.float64(0.5994550408719346), 'D_support': np.int64(332)}, 

In [101]:
                # Question: Where is a business restaurant likely to be located?
                #     A. corporate office clusters AND at university campuses
                #     B. business park entrances AND near conference venues
                #     C. NEITHER near conference venues NOR at university campuses
                #     D. NEITHER near conference venues NOR near tourist sites

                #     Answer: B

In [114]:
import os
import shutil

src = "/content"
dst = "/content/dev_results"

# Create destination folder if not exists
os.makedirs(dst, exist_ok=True)

# Loop through files in /content
for file in os.listdir(src):
    if file.lower().endswith(".csv") or file.lower().endswith(".json"):
        shutil.copy(os.path.join(src, file), dst)

print("Done! Copied all CSV and JSON files.")


Done! Copied all CSV and JSON files.


In [117]:
!zip -r dev_results.zip /content/dev_results


  adding: content/dev_results/ (stored 0%)
  adding: content/dev_results/csqa_2shot_MIXED_results.csv (deflated 76%)
  adding: content/dev_results/csqa_2shot_OR_results.csv (deflated 76%)
  adding: content/dev_results/csqa_1shot_AND_results.csv (deflated 76%)
  adding: content/dev_results/train_logical_combinations_output.json (deflated 91%)
  adding: content/dev_results/csqa_1shot_OR_results.csv (deflated 76%)
  adding: content/dev_results/dev_all_hf.json (deflated 73%)
  adding: content/dev_results/csqa_zeroshot_MIXED_results.csv (deflated 76%)
  adding: content/dev_results/csqa_zeroshot_NEITHER_results.csv (deflated 77%)
  adding: content/dev_results/test_all_hf.json (deflated 73%)
  adding: content/dev_results/csqa_2shot_NEITHER_results.csv (deflated 77%)
  adding: content/dev_results/train_all_hf.json (deflated 73%)
  adding: content/dev_results/logical_combinations_output.json (deflated 91%)
  adding: content/dev_results/csqa_3shot_NEITHER_results.csv (deflated 77%)
  adding: con

In [118]:
from google.colab import files
files.download("dev_results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')